In [1]:
import numpy as np
from scipy.spatial.distance import cdist
from SALib.sample.morris.strategy import Strategy
from SALib.sample.morris import generate_trajectory, _sample_oat

In [2]:
sample = _sample_oat({'num_vars': 10}, 5)

In [3]:
sample.shape

(55, 10)

In [4]:
sample[:11, 4]

array([0.66666667, 0.66666667, 0.66666667, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        ])

## `sample` matrix

The way that this array is constructed is... unintuituve. Maybe it is like this for backwards compatibility. `sample` is a two-dimensional array which includes three dimensions: trajectory, parameter, and parameter value. Personally, I would prefer this to be a three dimensional array (order doesn't really matter).

Note: My (older, unmaintained) [py-elem-effects](https://bitbucket.org/cmutel/py-elem-effects/) library doesn't store the coordinates of every single point at each point on the trajectory, but rather the starting and ending values for each parameter, and the order the parameters change in. This means you reduce memory usage substantially.

The rows are parameter values for the first trajectory, follow by for the second trajectory, etc. Columns are the parameters.

## Trajectory generation performance

The function `_sample_oat` includes this line:

    sample = np.array([generate_trajectory(group_membership, num_levels)
                       for n in range(N)])

These types of loops should almost always be avoided, as there are numpy operators that are much faster. Let's see if we can make trajectory generation faster.

In [5]:
m = np.asmatrix(np.identity(1000, dtype=int))

In [6]:
%timeit generate_trajectory(m, 4)

141 ms ± 11.9 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [7]:
type(generate_trajectory(m, 4))

numpy.matrixlib.defmatrix.matrix

In [10]:
%timeit generate_trajectory_2(1000, 4)

21.6 ms ± 3.95 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
